In [1]:
import tomotopy as tp
import pandas as pd

In [2]:
import nltk
from time import time
#nltk.download('punkt')
from nltk import word_tokenize 
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('words')
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import r2_score

## Caricamento del dataset

In [29]:
data_1 = pd.read_csv('C:/Users/micky/OneDrive/Desktop/Tesi DS/data/drugsComTest_raw.csv')
data_2 = pd.read_csv('C:/Users/micky/OneDrive/Desktop/Tesi DS/data/drugsComTrain_raw.csv')
data_1.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4


In [30]:
data = pd.concat([data_1,data_2])[['review','condition']]

In [31]:
print("Shape of the dataset: ", data.shape)

Shape of the dataset:  (215063, 2)


In [32]:
# sample 5% - create dataset Subsampling-1
data = data.sample(frac=.05, random_state=9, replace=False)
data.shape

(10753, 2)

In [33]:
# sample 20% - create dataset Subsampling-2
data = data.sample(frac=.2, random_state=9, replace=False)
data.shape

(2151, 2)

In [34]:
data = data[pd.notnull(data['review'])]

In [35]:
count_df = data[['condition', 'review']].groupby('condition').aggregate({
    'review': 'count'
}).reset_index().sort_values('review', ascending=False)
count_df.head()

,condition,review
39,Birth Control,373
76,Depression,145
178,Pain,85
21,Anxiety,79
8,Acne,69


In [46]:
target_conditions = count_df[count_df['review']>40]['condition'].values
target_conditions

array(['Birth Control', 'Depression', 'Pain', 'Anxiety', 'Acne',
       'Weight Loss', 'ADHD', 'Insomnia', 'Bipolar Disorde', 'Obesity',
       'Diabetes, Type 2'], dtype=object)

In [47]:
len(target_conditions)

11

In [48]:
data = data[data['condition'].apply(lambda x: True if x in target_conditions else False)]

In [49]:
data.shape #new shape

(1062, 2)

In [50]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['condition'] = le.fit_transform(data['condition'])

In [51]:
data.condition.unique()

array([ 4,  3, 10,  1,  5,  2,  8,  9,  7,  0,  6])

Conversione in minuscolo

In [52]:
data['review'] = data['review'].str.lower()

Rimozione numeri

In [53]:
data['review'] = data['review'].str.replace(r'[0-9]+', ' ', regex=True)

Rimozione link

In [54]:
data['review'] = data['review'].str.replace(r'http\S+', ' ', regex=True)

Rimozione punteggiatura

In [55]:
data['review'] = data['review'].str.replace(r'[^\w\s]', ' ', regex=True)

Rimozione caratteri speciali tranne le lettere

In [56]:
data['review'] = data['review'].str.replace(r'[^a-zA-Z0-9 ]', ' ', regex=True)

Rimozione lettere singole e doppie

In [57]:
data['review'] = data['review'].str.replace('\\b\\w{1,2}\\b', '', regex=True)

Rimozione spazi extra

In [58]:
data['review'] = data['review'].str.replace(' +', ' ', regex=True)

In [59]:
data['review']

145347     been taking this pill for almost weeks now fa...
85009      per day and honestly really dont know how muc...
97741      always research before putting anything your ...
131457     can really say that was nervous starting this...
110005     impossible keep weight off been period for mo...
                                ...                        
38976      been pristiq for week and already feel like n...
5048       tears joy omg thank you all started tramadol ...
116557     has done its job not pregnant have had since ...
60135      have had the implanon for almost year and lov...
153628     had panic attack and doctor diagnosed with ga...
Name: review, Length: 1062, dtype: object

Rimozione stopwords

In [60]:
stop = stopwords.words('english')
data['review'] = data['review'].apply(lambda x: [item for item in str(x).split() if item not in stop])

Stemming

In [61]:
stemmer = SnowballStemmer("english")
data['review'] = data['review'].apply(lambda x: [stemmer.stem(y) for y in x])
data['review']

145347    [take, pill, almost, week, far, noth, bad, thi...
85009     [per, day, honest, realli, dont, know, much, h...
97741     [alway, research, put, anyth, bodi, doctor, sa...
131457    [realli, say, nervous, start, pill, read, revi...
110005    [imposs, keep, weight, period, month, dri, ski...
                                ...                        
38976     [pristiq, week, alreadi, feel, like, new, pers...
5048      [tear, joy, omg, thank, start, tramadol, back,...
116557    [done, job, pregnant, sinc, jan, period, spora...
60135     [implanon, almost, year, love, random, period,...
153628    [panic, attack, doctor, diagnos, gad, general,...
Name: review, Length: 1062, dtype: object

Rimozione eventuali documenti vuoti

In [62]:
data = data[data['review'].map(len) != 0]

In [63]:
data.shape

(1062, 2)

### Train-test split

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
data.head()

,review,condition
145347,"[take, pill, almost, week, far, noth, bad, thi...",4
85009,"[per, day, honest, realli, dont, know, much, h...",3
97741,"[alway, research, put, anyth, bodi, doctor, sa...",4
131457,"[realli, say, nervous, start, pill, read, revi...",10
110005,"[imposs, keep, weight, period, month, dri, ski...",4


In [66]:
#train-test split 80%-20%
X_train, X_test, y_train, y_test = train_test_split(data['review'],
                                                    data['condition'],
                                                    test_size=0.2,
                                                    random_state=1234)

In [69]:
slda = tp.SLDAModel(k=10, vars=['l'], seed=1234, rm_top=10, min_df=3, alpha=.1)

for document, rating in zip(X_train, y_train):
    slda.add_doc(document, [rating])

slda.burn_in = 10
slda.train(0)

print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(slda.docs), len(slda.used_vocabs), slda.num_words))
print('Removed Top words: ', *slda.removed_top_words)

Num docs:849, Num Vocabs:1361, Total Words:28435
Removed Top words:  take month day year effect week get feel period start


In [70]:
t0 = time()
slda.train(100)
print("done in %0.3fs." % (time() - t0))

done in 0.395s.


In [71]:
yhat = []
for document in X_test:
    doc = slda.make_doc(document)
    slda.infer(doc)
    yhat.append(slda.estimate(doc)[0])
rsq = r2_score(y_test, yhat)
print('R2 score sul test set per k={}: {}'.format(slda.k, rsq))

R2 score sul test set per k=10: 0.3204473245446223


In [72]:
for i in range(4):
    print(slda.get_topic_words(i, 30), "\n")

[('pill', 0.07518204301595688), ('control', 0.05325477197766304), ('gain', 0.044142141938209534), ('birth', 0.04214875400066376), ('first', 0.03844675049185753), ('weight', 0.0358838252723217), ('would', 0.024777809157967567), ('swing', 0.02335396036505699), ('mood', 0.023069189861416817), ('use', 0.02164534293115139), ('even', 0.015949947759509087), ('switch', 0.014526099897921085), ('sex', 0.014241330325603485), ('back', 0.013956560753285885), ('good', 0.01367179024964571), ('headach', 0.01338702067732811), ('well', 0.01281748153269291), ('almost', 0.012247942388057709), ('love', 0.011678403243422508), ('drive', 0.011678403243422508), ('breast', 0.011108864098787308), ('extrem', 0.010824094526469707), ('regular', 0.010254554450511932), ('someth', 0.009969784878194332), ('definit', 0.009969784878194332), ('alway', 0.009685015305876732), ('actual', 0.009400245733559132), ('acn', 0.009115476161241531), ('worri', 0.009115476161241531), ('emot', 0.008545937016606331)] 

[('pain', 0.043012